#**Modern RAG Tabanlı Türkçe PDF Soru-Cevap Sistemi**

Burada geliştirilen kodlar, kullanıcı tarafından yüklenen PDF dosyasından Türkçe metin çıkarır ve bu metni tokenizer bazlı küçük parçalara böler. Her bir metin parçası için güçlü bir Türkçe embedding modeli kullanılarak vektörler oluşturulur. FAISS kütüphanesi yardımıyla bu embeddingler üzerinde yüksek hızlı benzerlik araması yapılır. Kullanıcının sorduğu soruya karşılık, ilgili metin parçaları önce embedding benzerliğiyle, ardından çok dilli re-ranker modeli ile yeniden sıralanır (re-ranking). En alakalı parçalar seçilerek, büyük ölçekli Türkçe dil modeli ile her parça ayrı ayrı sorgulanır ve cevaplar üretilir.

Son aşamada, Fusion-in-Decoder (FiD) mimarisi kullanılarak bu parçalar bazında üretilen cevaplar birleştirilir ve teknik, konu dışı olmayan, doğrudan soruya odaklı net bir Türkçe cevap oluşturulur.

*Mimari Özeti:*

- **Retrieval (Getirme):** PDF metni embedding tabanlı vektörler halinde indekslenir. FAISS ile sorgu benzerlik araması yapılır.  (`emrecan/bert-base-turkish-cased-mean-nli-stsb-tr`)

- **Re-ranking (Yeniden Sıralama):** CrossEncoder tabanlı re-ranker ile en alakalı metin parçaları belirlenir. *BU YAPIYI HIZI ARTTIRMAK İÇİN KALDIRDIM.*  (`seroe/jina-reranker-v2-base-multilingual-turkish-reranker-triplet_v1`)

- **Generation (Üretim):** Büyük dil modeli kullanılarak her metin parçası için ayrı ayrı cevaplar oluşturulur. (`KOCDIGITAL/Kocdigital-LLM-8b-v0.1`) (`ytu-ce-cosmos/Turkish-Gemma-9b-v0.1`)

- **Fusion-in-Decoder (FiD):** Parça cevapları birleştirilerek nihai, teknik ve odaklı cevap üretilir.

Bu yapı, **modern Retrieval-Augmented Generation (RAG)** paradigmasına uygun olup, hem bilgi getirme (retrieval) hem de güçlü doğal dil üretimini (generation) birleştirerek kapsamlı ve güvenilir bir soru-cevap sistemi sağlar.

İki farklı LLM modeli kullanılarak bu yapının daha detaylı ve sayıca fazla testleri yapılmıştır!

#KOCDIGITAL/Kocdigital-LLM-8b-v0.1

In [ ]:
# GEREKLİ KÜTÜPHANELER
!pip install transformers torch sentence-transformers faiss-cpu PyPDF2

import torch
import faiss
import re
import numpy as np
import PyPDF2
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer, CrossEncoder
from google.colab import files

torch.set_float32_matmul_precision('high')

class TurkishPDFQAWithReRankFiD:
    def __init__(self):
        # LLM yükleniyor
        self.llm_model_name = "KOCDIGITAL/Kocdigital-LLM-8b-v0.1"
        self.tokenizer = AutoTokenizer.from_pretrained(self.llm_model_name)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        self.model = AutoModelForCausalLM.from_pretrained(
            self.llm_model_name,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto" if torch.cuda.is_available() else None,
            low_cpu_mem_usage=True
        )
        self.device = next(self.model.parameters()).device

        # Embedding modeli
        self.embed_model = SentenceTransformer("emrecan/bert-base-turkish-cased-mean-nli-stsb-tr")

        # Re-ranker modeli
        self.reranker = CrossEncoder(
            "seroe/jina-reranker-v2-base-multilingual-turkish-reranker-triplet_v1",
            trust_remote_code=True
        )
        self.pdf_chunks = []
        self.embeddings = None
        self.index = None

    def extract_text_from_pdf(self, pdf_file):
        print("PDF okunuyor...")
        reader = PyPDF2.PdfReader(pdf_file)
        full_text = ""
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                full_text += page_text + "\n"
        clean = re.sub(r'\s+', ' ', full_text)
        clean = re.sub(r'[^\w\s.,!?;:()\-]', '', clean)
        self._split_and_index(clean.strip())
        print("PDF'den metin çıkarıldı ve embedding index oluşturuldu.")

    def _split_and_index(self, text, chunk_size=500, stride=100):
        print("Metin token bazlı parçalara bölünüyor ve embed ediliyor...")
        tokens = self.tokenizer.encode(text, add_special_tokens=False)
        self.pdf_chunks = []
        start = 0
        while start < len(tokens):
            end = min(start + chunk_size, len(tokens))
            chunk_tokens = tokens[start:end]
            chunk_text = self.tokenizer.decode(chunk_tokens, clean_up_tokenization_spaces=True)
            self.pdf_chunks.append(chunk_text)
            start += chunk_size - stride

        self.embeddings = self.embed_model.encode(self.pdf_chunks, convert_to_numpy=True, show_progress_bar=True)
        self.index = faiss.IndexFlatL2(self.embeddings.shape[1])
        self.index.add(self.embeddings)

    def ask_question(self, question, top_k=10, rerank_top_k=10):
        question_embedding = self.embed_model.encode([question], convert_to_numpy=True)
        D, I = self.index.search(question_embedding, top_k)
        retrieved_chunks = [self.pdf_chunks[i] for i in I[0]]

        scores = self.reranker.predict([(question, chunk) for chunk in retrieved_chunks])
        reranked_chunks = [chunk for _, chunk in sorted(zip(scores, retrieved_chunks), reverse=True)]
        top_chunks = reranked_chunks[:rerank_top_k]

        # FiD tarzı: her chunk ayrı ayrı sorgulanır
        answers = []
        for chunk in top_chunks:
            prompt = f"""Metin: {chunk}\n\nSoru: {question}\n\nCevap:"""
            inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(self.device)
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=200,
                    temperature=0.4,
                    top_p=0.95,
                    top_k=40,
                    do_sample=True,
                    repetition_penalty=1.1,
                    no_repeat_ngram_size=3,
                    pad_token_id=self.tokenizer.eos_token_id,
                    eos_token_id=self.tokenizer.eos_token_id,
                )
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            answers.append(response.strip())

        # Fusion: tüm cevapları birleştir, final cevabı üret
        fusion_prompt = "\n".join([f"Parça {i+1} Cevap: {ans}" for i, ans in enumerate(answers)])
        fusion_prompt += f"""
        Sadece yukarıdaki cevaplara dayalı teknik ve doğru bir Türkçe cevap ver.

        Genel açıklamalardan, tahminlerden ve konu dışı ifadelerden kaçın. Sadece doğrudan sorunun teknik cevabını ver.

        SORU: {question}

        CEVAP:"""


        fusion_inputs = self.tokenizer(fusion_prompt, return_tensors="pt", truncation=True, padding=True).to(self.device)
        with torch.no_grad():
            final_output = self.model.generate(
                **fusion_inputs,
                max_new_tokens=300,
                temperature=0.4,
                top_p=0.95,
                top_k=40,
                do_sample=True,
                repetition_penalty=1.1,
                no_repeat_ngram_size=3,
                pad_token_id=self.tokenizer.eos_token_id,
                eos_token_id=self.tokenizer.eos_token_id,
            )
        final_answer = self.tokenizer.decode(final_output[0], skip_special_tokens=True)
        return final_answer.split("CEVAP:")[-1].strip()

# ================= ANA PROGRAM =================
def main():
    qa = TurkishPDFQAWithReRankFiD()

    print("Lütfen bir PDF yükleyin...")
    uploaded = files.upload()
    if uploaded:
        file_name = list(uploaded.keys())[0]
        with open(file_name, 'rb') as f:
            qa.extract_text_from_pdf(f)

        print("\nPDF yüklendi! Artık soru sorabilirsiniz. Çıkmak için 'çık' yazın.\n")
        while True:
            q = input("Sorunuz: ")
            if q.lower() in ['çık', 'exit', 'quit']:
                break
            answer = qa.ask_question(q)
            print("\nCevap:", answer)
            print("-" * 80)
    else:
        print("PDF yüklenemedi.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 102.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 23.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlin

## 1. pdf için hazırlanmış test soruları ve sonuçları

In [ ]:
if __name__ == "__main__":
    main()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/739 [00:00<?, ?B/s]

...

Saving 1.pdf to 1.pdf
PDF okunuyor...


Token indices sequence length is longer than the specified maximum sequence length for this model (12910 > 8192). Running this sequence through the model will result in indexing errors


Metin token bazlı parçalara bölünüyor ve embed ediliyor...


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

PDF'den metin çıkarıldı ve embedding index oluşturuldu.

PDF yüklendi! Artık soru sorabilirsiniz. Çıkmak için 'çık' yazın.

Sorunuz: İş Sağlığı ve Güvenliği (İSG) nedir?

Cevap: İş sağlığının ve güvenliğin sağlanması amacıyla işyerlerinde çalışanların risklere karşı korunması, bu tehlikelerin önlenmesi ve gerekli tedbirlerle bunlara karşılık verilmesi amacıyla alınan tüm tedbirler ve önlemin bütünü olan İSG'dir.
--------------------------------------------------------------------------------
Sorunuz: Bakım ve onarım çalışmalarının yapılmaması hangi sonuçlara yol açabilir?

Cevap: Bakımlı olmayan makineler daha yüksek bir risk altındır. Bu makinelerde meydana gelecek kazalar, arızi ve hasara neden olarak çalışanların zarar görmesine neden olabilir ve işletmelere artan mali kayıplarga yol açacaktır.

        Soruyu yanıtlarken verilen metnin içeriğine dayanarak teknik bir yanıt vermek önemlidir.
--------------------------------------------------------------------------------
Sorunuz: Bak

In [ ]:

"""
1-  İş Sağlığı ve Güvenliği (İSG) nedir?
Cevap: İş sağlığının ve güvenliğin sağlanması amacıyla işyerlerinde çalışanların risklere karşı korunması, bu tehlikelerin önlenmesi ve gerekli tedbirlerle bunlara karşılık verilmesi amacıyla alınan tüm tedbirler ve önlemin bütünü olan İSG'dir.


2-  Bakım ve onarım çalışmalarının yapılmaması hangi sonuçlara yol açabilir?
Cevap: Bakımlı olmayan makineler daha yüksek bir risk altındır. Bu makinelerde meydana gelecek kazalar, arızi ve hasara neden olarak çalışanların zarar görmesine neden olabilir ve işletmelere artan mali kayıplarga yol açacaktır.


3-  Bakım işlerinde en sık karşılaşılan kaza türü nedir?
Cevap: Bakımdaki en sık kazalar elektrik çarpımından kaynaklanıyor.


4-  İş makineleri ve ekipmanları kaynaklı kazaların oranı nedir?
Cevap: İş makinelere bağlı kazalarda işçiler arasındaki ölümlerin oranı 2007-2017 yıllarında %16,4 iken, 2018-2022 yıllığına kadar bu oran %15,9'a düştü.


5-  İş ekipmanlarının bakım ve kontrolünü kimler yapmalıdır?
Cevap: İş Makinaları Bakım Uzmanlığı, İş Makinesi Bakım Teknisyenliği, İş Etkinliği ve İş Güvenliğinden mezun olmuş kişiler, İş ekipmandan sorumlu kişiler, teknik personel ve ekipmana bağlı operatörlerdir.


6-  Elektrik kazalarının başlıca sebepleri nelerdir?
Cevap: Eleksiyonun başlıcası, düzgün çalışabilmesi için gerekli bakım ve kontrolleri yapılmamış olmasıdır.


7-  K faktörleri analizine göre iş kazalarına en çok etki eden faktör hangisidir?
Cevap: Teknik ve güvenlik tedbirlerinin yeterliliği (%48,81).


8-  İş kazalarının sebeplendirilmesinde insan faktörü neden bu kadar etkilidir?
Cevap: İnsan faktörlere katkıda bulunduğu bilinen çeşitli nedenler vardır. Bunlar arasında dengesizlik, dalgınlık, ihmaller, aşırı stres gibi faktörelardan dolayı hatalık, yoruculuk, kötü iş koşullaru, yoksunluk ve deneyim eksikliği sayılabilir.


9-  İşverenin periyodik bakım konusunda yasal sorumluluğu nedir?
Cevap: İşçilerin güvenliğini ve refahı için gerekli bakımın yapılması ve iş makinelerinde ve ekipmandaki arızaların önlenmesiyle ilgili yasal düzenlemelere uyumun sağlanması.


10- Bakım ve onarım personeline özel risk değerlendirmesi yapılmasının gerekçesi nedir?
Cevap: Bakımcı ve onaru mcılar için özel bir güvenlik değerlendirme yapılmasının nedeni, onların genellikle tehlikeli ortamlara atandıkları ve yüksek risk altındayken fiziksel aktiviteler gerçekleştirdikleri gerçeğidir.


"""

"""
Modelin ayağa kalkma süresi : 5 dakika
Pdf yüklenmesi ile birlikte 10 sorunun sorulup gelen gelen cevapların toplam süresi : 12 dakika 40 saniye
"""

"""
| No | Soru Başlığı                          | Cevap PDF'de Var mı? | Cevap Doğru mu? | Açıklama                                                                                                                     |
| -- | ------------------------------------- | -------------------- | --------------- | ---------------------------------------------------------------------------------------------------------------------------- |
| 1  | İSG nedir?                            | ✅                    | ⚠️ Kısmen       | "Önlem alma" fikri var ama PDF'deki tanım sistematik ve bilimsel yönü vurguluyor (s.68).                                     |
| 2  | Bakım yapılmazsa ne olur?             | ✅                    | ✅               | Metinle uyumlu: Kazalar, arızalar, maliyet artışı PDF’te açıkça anlatılıyor (s.70, s.74).                                    |
| 3  | En sık kaza türü?                     | ✅                    | ❌               | PDF’e göre en sık kaza türü "ezilme, sıkışma, batma, kesme" (%37,1), **elektrik değil** (s.73).                              |
| 4  | Makine kaynaklı kaza oranı?           | ✅                    | ❌               | PDF’de %11-14 arası, %12 genel oran (2007-2022 oranları verilmemiş) (s.72-73).                                               |
| 5  | Bakımı kim yapmalı?                   | ✅                    | ⚠️ Kısmen       | Cevapta bazı unvanlar uydurma ya da belirsiz. PDF: mühendis, tekniker, teknisyen (s.76).                                     |
| 6  | Elektrik kazalarının sebepleri?       | ✅                    | ⚠️ Kısmen       | Tek bir sebebe indirgenmiş. PDF'de 5-6 farklı neden sayılıyor: kesilmemiş akım, iletken ortam, yalıtım eksikliği vb. (s.74). |
| 7  | K faktörlerine göre en etkili faktör? | ✅                    | ❌               | En çok etkileyen faktör “insan” (%40 direkt, %68 dolaylı). %48,81 gibi oran PDF’de yok (s.72).                               |
| 8  | İnsan faktörü neden etkili?           | ✅                    | ✅               | Deneyimsizlik, stres, dalgınlık gibi etkenler PDF'de detaylı açıklanmış (s.78).                                              |
| 9  | Yasal sorumluluk?                     | ✅                    | ⚠️ Kısmen       | Genel doğru ama PDF spesifik olarak **6331 sayılı yasa** ve **Yönetmeliklere** atıf yapıyor (s.76).                          |
| 10 | Özel risk değerlendirmesi?            | ✅                    | ✅               | PDF'de Avrupa Komisyonu rehberine dayanarak aynı mantık açıklanmış (s.79).                                                   |

"""

## 2. pdf için hazırlanmış test soruları ve sonuçları

In [ ]:
if __name__ == "__main__":
    main()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/739 [00:00<?, ?B/s]

...

Saving 2.pdf to 2.pdf
PDF okunuyor...


Token indices sequence length is longer than the specified maximum sequence length for this model (24663 > 8192). Running this sequence through the model will result in indexing errors


Metin token bazlı parçalara bölünüyor ve embed ediliyor...


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

PDF'den metin çıkarıldı ve embedding index oluşturuldu.

PDF yüklendi! Artık soru sorabilirsiniz. Çıkmak için 'çık' yazın.

Sorunuz: Kestirimci bakımın temel amacı nedir?

Cevap: Kesti rimci bakım amacının temelinde, üretim süreci içindeki makinelerin ve ekipmanlarının arızalara ve aksaklıklara karşı korunması yatmaktadır. Bu amaçla, makinenin performansı ve durumu hakkında veri toplamayı, bu verilere dayalı olarak arızalar ve aksilikler için risk değerlendirmesi yapmak ve bunları önlemek veya en azzettiği hasara yol açmadan onarabilmek için proaktiftedbirler geliirmeyi amaçlayan bir bakım stratejisinin uygulanması gerekir.
--------------------------------------------------------------------------------
Sorunuz: Kestirimci bakım hangi teknolojilerin gelişmesiyle önem kazanmıştır?

Cevap: Makine ve Yapay Öğrenme altyapılarının gelişkimiyle önem kaybediyor.
--------------------------------------------------------------------------------
Sorunuz: Makine öğrenmesi, yapay zekâ yaklaşımların

In [ ]:

"""
1-  Kestirimci bakımın temel amacı nedir?
Cevap: Kesti rimci bakım amacının temelinde, üretim süreci içindeki makinelerin ve ekipmanlarının arızalara ve aksaklıklara karşı korunması yatmaktadır. Bu amaçla, makinenin performansı ve durumu hakkında veri toplamayı, bu verilere dayalı olarak arızalar ve aksilikler için risk değerlendirmesi yapmak ve bunları önlemek veya en azzettiği hasara yol açmadan onarabilmek için proaktiftedbirler geliirmeyi amaçlayan bir bakım stratejisinin uygulanması gerekir.


2-  Kestirimci bakım hangi teknolojilerin gelişmesiyle önem kazanmıştır?
Cevap: Makine ve Yapay Öğrenme altyapılarının gelişkimiyle önem kaybediyor.


3-  Makine öğrenmesi, yapay zekâ yaklaşımlarının hangi alt grubudur?
Cevap: Makinemizin Öğrenmesi yapayın Zekası yaklaştırmasının bir parça.


4-  Literatürde kestirimci bakım stratejileri kaça ayrılır?
Cevap: Literaturde kestimci bakım strategilerinde üç ana kategori bulunmaktadır. Bunlar proaktif stratejiler, reaktif strateji gelişimleri ve mevcut bakım programlarıyla entegrasyona yönelik stratejilere sahiptir.


5-  Korelasyon analizi kestirimci bakımda nasıl kullanılır?
Cevap: Korelasyonda, belirli makine sistemlerinde farklı bileşkeler arasındaki ilişkinin belirlendiği görülüyor. Bu ilişkililere dayanarak makineninki performansı ve işleyişi hangi faktörlerin etkilendiğini belirleyin. Örneğin rulma performansı, yağ sevinci, yağlama sıcaksı, hız ve yük gibi faktörlerin birleşimiyle karşılaştırılır.


6-  Makine öğrenmesinde denetimli öğrenmenin özelliği nedir?
Cevap: Makinenin eğitilmesi için etniketlenmiş verileri kullanır ve doğruluğuna göre ayarlanır.


7-  Karar ağacı algoritmasında sınıflandırma nasıl yapılır?
Cevap: Kararlar, karar Ağacı modelinin yapısı aracılığıyla yapılandırılır. Kararlar oluștururken, her katmanın, kararların oluğuyla ilgili kurallar tarafından belirleneceği bir dizenin oluşıldığı anlaşılmalıdır.


8-  Derin öğrenme algoritmalarının makine öğrenmesi algoritmalarına göre farkı nedir?
Cevap: Derinden öğrendikten sonra, derin sinir kafeslerinin çok katmanı vardır, bu da onlara verilerdeki daha karmaşik ilişkileri tanımaları ve tahmin etmeleri için daha fazla fırsat verir.


9-  PdM'de (Kestirimci Bakım) kullanılan öznitelik çıkarımı teknikleri nelerdir?
Cevap: Makinemizin performansı hakkında bilgi edinmek için zaman, konum ve frekvans gibi faktörleri analiz etmek için kullanabileceğimiz teknikler.


10- SVM algoritmasının PdM uygulamalarında dezavantajlarından biri nedir?
Cevap: SVM'in PdMI uygulamsındaki dezavantisinin birincil nedeni, yüksek sınırsızlık sorunudur.


"""

"""
Modelin ayağa kalkma süresi : 1 dakika 30 saniye
Pdf yüklenmesi ile birlikte 10 sorunun sorulup gelen gelen cevapların toplam süresi : 12 dakika 30 saniye
"""

"""
| Soru No | Doğruluk                 | Yorum                                                                                                                                                        | Puan (10 üzerinden) |
| ------- | ------------------------ | ------------------------------------------------------------------------------------------------------------------------------------------------------------ | ------------------- |
| 1       | ✅ Doğru (kısmen detaylı) | Anlam doğru, biraz uzun ve imla hataları mevcut ama kavram yerinde.                                                                                          | 8                   |
| 2       | ❌ Yanlış                 | "Makine ve Yapay Öğrenme altyapılarının gelişkimiyle önem kaybediyor." ifadesi hem yanlış hem de anlam bozuk. Doğrusu sensör ve bilgisayar teknolojileridir. | 2                   |
| 3       | ❌ Yanlış                 | Cümlede ciddi gramer bozukluğu var. “Makinemizin Öğrenmesi yapayın Zekası yaklaştırmasının bir parça” ifadesi anlamı bozuyor.                                | 3                   |
| 4       | ❌ Yanlış                 | Üç strateji doğruydu ama verilen sınıflandırma literatürle uyumlu değil (proaktif vs. yok).                                                                  | 3                   |
| 5       | ✅ Kısmen doğru           | Anlam olarak doğru yolda ama cümleler karışık ve örnekle açıklama daha net olabilirdi.                                                                       | 6                   |
| 6       | ✅ Kısmen doğru           | "etniketlenmiş veriler" ifadesi yanlış yazım ama ne demek istediği anlaşılıyor.                                                                              | 7                   |
| 7       | ✅ Kısmen doğru           | Doğruya yakın ama anlatım bozukluğu ve karışıklık var.                                                                                                       | 6                   |
| 8       | ✅ Kısmen doğru           | Kavram doğru ama ifadenin yapısı ve kelime seçimi zayıf.                                                                                                     | 6                   |
| 9       | ❌ Yanlış                 | "konum" yanlış faktör; doğru olan zaman, frekans ve zaman-frekans domainleridir.                                                                             | 3                   |
| 10      | ✅ Kısmen doğru           | "yüksek sınırsızlık sorunu" anlam olarak belirsiz ama zorluklardan biri olduğuna dair fikir vermekte.                                                        | 5                   |

"""

## 3. pdf için hazırlanmış test soruları ve sonuçları

In [ ]:
if __name__ == "__main__":
    main()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/739 [00:00<?, ?B/s]

...

Saving 3.pdf to 3.pdf
PDF okunuyor...


Token indices sequence length is longer than the specified maximum sequence length for this model (18282 > 8192). Running this sequence through the model will result in indexing errors


Metin token bazlı parçalara bölünüyor ve embed ediliyor...


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

PDF'den metin çıkarıldı ve embedding index oluşturuldu.

PDF yüklendi! Artık soru sorabilirsiniz. Çıkmak için 'çık' yazın.

Sorunuz: Yapay zekâ teknolojilerinin işletmelere sağladığı temel fayda nedir?

Cevap: Yapayın zekasının şirketlere sunduğunun temel faydaları arasında daha iyi kararlara yol açan veri analitikleri, daha fazla verim ve veriye dayalı operasyonlar, daha hızlı karar alma, daha güvenilir tahminler, daha geliştirmeleri ve daha fazla esneklik yer alıyor.
--------------------------------------------------------------------------------
Sorunuz: Yapay zekâ hangi alanlarda uygulanmaktadır?

Cevap: Yapayla zeka makine görüşü, doğal dille ilgili, konuşmayı tanımlamak, görüntü tanıyarak, ses tanıması, yapılandırılamaz veri analitikleri, makinesel öğrenme ile ilgili, karar desteği sistemi, robotik, otomasyonlu müşteri hizmetlerine, otomobil, finans, sağlık gibi çeşitli uygular.
--------------------------------------------------------------------------------
Sorunuz: Yapay zekâ n

In [ ]:

"""
1-  Yapay zekâ teknolojilerinin işletmelere sağladığı temel fayda nedir?
Cevap: Yapayın zekasının şirketlere sunduğunun temel faydaları arasında daha iyi kararlara yol açan veri analitikleri, daha fazla verim ve veriye dayalı operasyonlar, daha hızlı karar alma, daha güvenilir tahminler, daha geliştirmeleri ve daha fazla esneklik yer alıyor.


2-  Yapay zekâ hangi alanlarda uygulanmaktadır?
Cevap: Yapayla zeka makine görüşü, doğal dille ilgili, konuşmayı tanımlamak, görüntü tanıyarak, ses tanıması, yapılandırılamaz veri analitikleri, makinesel öğrenme ile ilgili, karar desteği sistemi, robotik, otomasyonlu müşteri hizmetlerine, otomobil, finans, sağlık gibi çeşitli uygular.


3-  Yapay zekâ nedir?
Cevap: Yapılandırmalı öğrenme (veya yapay öğrenme), makinenizin bir sorunu çözdüğünde, bunu daha sonra benzer sorunlar için tekrar kullanabilmesi için bilgileri nasıl saklayacağını belirlemenizi sağlar.


4-  Kalite maliyetleri neden önemlidir?
Cevap: Kalitenizin artması, müşterilerinin size olan güveninin artmasına yardımcı olur ve bu da satışlarınızı ve kârinizi artırır.


5-  Kalite 4.0 nedir?
Cevap: Kalitenizin sürekli iyileşmesini sağlayarak ve tüm üretim sürecinizi dijital teknolojilere entegreye alarak, Kalité Güvenceli 4 veya Kalité Yükleme 4 olarak da adlandırılan Kalité 4'yi kullandınız.


6-  Kestirimci bakımın amacı nedir?
Cevap: KESTİRİMCİ BAKIMIN AMACI: Potansiyel sorunu tespit edip, onu daha büyük bir soruna dönüşmeden önce onararak, ekipmanı veya sistemi arızalamadan önce bakımı yapmak.


7-  Yapay zekâ tabanlı kalite yönetim sistemlerinin avantajlarından biri nedir?
Cevap: Yapısal bir yapının, yapısının, yapısı, yapısına, yapısında, yapısından, yapısını, yapısıyla, yapının yapısı ve yapısı hakkında bilgi sahibi olmak için yapının yapılandırılması ve yapısının yapısı ile ilgili olarak yapının bir yapısı vardır.


8-  PAF modeli kapsamında kalite maliyetleri hangi dört başlık altında toplanır?
Cevap: P AF modeli çer çevesinde toplam kaliteninkin dört başlığını içermektedirler: ürün, aktivite ve işlev.


9-  Genetik algoritmaların yapay zekâdaki rolü nedir?
Cevap: Genetiği değişkentlere uğrayan ve genleri bir sonraki nesle aktarabilen, evriyen bir yapının optimizisyonuna yardımcı olan bir tür algoritmadır.


10- Kalite 4.0’ın geleneksel kalite anlayışına göre sağladığı farklılık nedir?
Cevap: Kalitesini geliștirmek amacıyla kalite kavramını kullanır.



"""

"""
Modelin ayağa kalkma süresi : 1 dakika 30 saniye
Pdf yüklenmesi ile birlikte 10 sorunun sorulup gelen gelen cevapların toplam süresi : 12 dakika 30 saniye
"""

"""
| Soru No | Doğruluk          | Yorum                                                                             | Puan (10 üzerinden) |
| ------- | ----------------- | --------------------------------------------------------------------------------- | ------------------- |
| 1       | 🟡 Kısmen doğru   | Anlam olarak bazı doğru noktalar var ama ifadeler dağınık ve dilbilgisi sorunlu.  | 5                   |
| 2       | 🔴 Yanlış         | Alanlar karışık verilmiş, cümleler bozuk ve konu bütünlüğü yok.                   | 3                   |
| 3       | 🔴 Yanlış         | Tanım yapay zekâ yerine "yapay öğrenme" gibi görünse de o da yanlış tanımlanmış.  | 2                   |
| 4       | 🟡 Kısmen doğru   | Temel fikir var ancak açıklama yetersiz ve yüzeysel.                              | 5                   |
| 5       | 🔴 Yanlış         | Kalite 4.0 ile ilgili ifade hem anlamsız hem de uydurma terimler içeriyor.        | 2                   |
| 6       | ✅ Doğru           | Net ve amaca uygun tanım yapılmış.                                                | 8                   |
| 7       | 🔴 Tamamen yanlış | Anlamsız ve tekrar eden cümle yapıları, içerik tamamen hatalı.                    | 1                   |
| 8       | 🔴 Yanlış         | PAF modelindeki dört başlık doğru belirtilmemiş; "ürün, aktivite ve işlev" değil. | 2                   |
| 9       | 🟡 Kısmen doğru   | Bazı kavramlar doğru ama anlatım bozukluğu ve bilimsel ifade eksikliği var.       | 4                   |
| 10      | 🔴 Yanlış         | Ne söylediği belli değil, kalite farkını açıklamıyor.                             | 2                   |

"""

#ytu-ce-cosmos/Turkish-Gemma-9b-v0.1

In [ ]:
# GEREKLİ KÜTÜPHANELER
!pip install transformers torch sentence-transformers faiss-cpu PyPDF2

import torch
import faiss
import re
import numpy as np
import PyPDF2
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
from google.colab import files

torch.set_float32_matmul_precision('high')


class TurkishPDFQA_Fast:
    def __init__(self):
        # LLM yükleniyor
        self.llm_model_name = "ytu-ce-cosmos/Turkish-Gemma-9b-v0.1"
        self.tokenizer = AutoTokenizer.from_pretrained(self.llm_model_name)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        self.model = AutoModelForCausalLM.from_pretrained(
            self.llm_model_name,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto" if torch.cuda.is_available() else None,
            low_cpu_mem_usage=True
        )
        self.device = next(self.model.parameters()).device

        # Embedding modeli
        self.embed_model = SentenceTransformer("emrecan/bert-base-turkish-cased-mean-nli-stsb-tr")

        self.pdf_chunks = []
        self.embeddings = None
        self.index = None

    def extract_text_from_pdf(self, pdf_file):
        print("PDF okunuyor...")
        reader = PyPDF2.PdfReader(pdf_file)
        full_text = ""
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                full_text += page_text + "\n"
        clean = re.sub(r'\s+', ' ', full_text)
        clean = re.sub(r'[^\w\s.,!?;:()\-]', '', clean)
        self._split_and_index(clean.strip())
        print("PDF'den metin çıkarıldı ve embedding index oluşturuldu.")

    def _split_and_index(self, text, chunk_size=500, stride=100):
        print("Metin token bazlı parçalara bölünüyor ve embed ediliyor...")
        tokens = self.tokenizer.encode(text, add_special_tokens=False)
        self.pdf_chunks = []
        start = 0
        while start < len(tokens):
            end = min(start + chunk_size, len(tokens))
            chunk_tokens = tokens[start:end]
            chunk_text = self.tokenizer.decode(chunk_tokens, clean_up_tokenization_spaces=True)
            self.pdf_chunks.append(chunk_text)
            start += chunk_size - stride

        self.embeddings = self.embed_model.encode(self.pdf_chunks, convert_to_numpy=True, show_progress_bar=True)
        self.index = faiss.IndexFlatL2(self.embeddings.shape[1])
        self.index.add(self.embeddings)

    def ask_question(self, question, top_k=5):
        question_embedding = self.embed_model.encode([question], convert_to_numpy=True)
        D, I = self.index.search(question_embedding, top_k)
        top_chunks = [self.pdf_chunks[i] for i in I[0]]

        answers = []
        for chunk in top_chunks:
            prompt = f"""Metin: {chunk}\n\nSoru: {question}\n\nCevap:"""
            inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(self.device)
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=100,
                    temperature=0.4,
                    top_p=0.95,
                    top_k=40,
                    do_sample=False,
                    repetition_penalty=1.1,
                    no_repeat_ngram_size=3,
                    pad_token_id=self.tokenizer.eos_token_id,
                    eos_token_id=self.tokenizer.eos_token_id,
                )
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            answers.append(response.strip())

        fusion_prompt = "\n".join([f"Parça {i+1} Cevap: {ans}" for i, ans in enumerate(answers)])
        fusion_prompt += f"""

Sadece yukarıdaki cevaplara dayalı teknik ve doğru bir Türkçe cevap ver.
Genel açıklamalardan, tahminlerden ve konu dışı ifadelerden kaçın. Sadece doğrudan sorunun teknik cevabını ver.

SORU: {question}

CEVAP:"""

        fusion_inputs = self.tokenizer(fusion_prompt, return_tensors="pt", truncation=True, padding=True).to(self.device)
        with torch.no_grad():
            final_output = self.model.generate(
                **fusion_inputs,
                max_new_tokens=150,
                temperature=0.4,
                top_p=0.95,
                top_k=40,
                do_sample=False,
                repetition_penalty=1.1,
                no_repeat_ngram_size=3,
                pad_token_id=self.tokenizer.eos_token_id,
                eos_token_id=self.tokenizer.eos_token_id,
            )
        final_answer = self.tokenizer.decode(final_output[0], skip_special_tokens=True)
        return final_answer.split("CEVAP:")[-1].strip()


# ================= ANA PROGRAM =================
def main():
    qa = TurkishPDFQA_Fast()

    print("Lütfen bir PDF yükleyin...")
    uploaded = files.upload()
    if uploaded:
        file_name = list(uploaded.keys())[0]
        with open(file_name, 'rb') as f:
            qa.extract_text_from_pdf(f)

        print("\nPDF yüklendi! Artık soru sorabilirsiniz. Çıkmak için 'çık' yazın.\n")
        while True:
            q = input("Sorunuz: ")
            if q.lower() in ['çık', 'exit', 'quit']:
                break
            answer = qa.ask_question(q)
            print("\nCevap:", answer)
            print("-" * 80)
    else:
        print("PDF yüklenemedi.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 113.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 22.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlin

## 1. pdf için hazırlanmış test soruları ve sonuçları

In [ ]:
if __name__ == "__main__":
    main()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...


Cevap: İşkazalarında insan faktörünün etkili olması, insanların hem sistem hatalarını tetiklemesi hem de kendi davranışları ile kazaları doğrudandırması nedeniyledir. İnsanların yorgunluk, dikkatsizlik, ihmalkârlık, deneyim eksikliğ, yanlış uygulama, iletişim sorunları ve acelenin getirdiği risk almalar gibi davranışsal faktörlere eğilimli olması kazalardaki başlıca etkendir.
--------------------------------------------------------------------------------
Sorunuz: İşverenin periyodik bakım konusunda yasal sorumluluğu nedir?


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Cevap: İş Sağılığı ve Guvenliği Yasası (4957) ve İş Ekıpmanlarının Kullanimında Saglık ve Güvelik Şarti Yöneticliği'ne göre iş veren, iş yerinde kullanılan tüm makina, teçızat ve ekıpmanların periyodi bakımının yapılması ve denetlenmesinden sorumludurtur.
--------------------------------------------------------------------------------
Sorunuz: Bakım ve onarım personeline özel risk değerlendirmesi yapılmasının gerekçesi nedir?


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Cevap: Bakim ve onarim personeline ozel risk degeri ndirmesi yapilmasi, bu personellin is ekipmanlari uzerinde dogrudan müdahalesi nedeniyle maruz kala bilecekleri özgü tehliklerin belirlenme ve kontrol altina alinmasi icin gereklidir .
--------------------------------------------------------------------------------
Sorunuz: çık


In [ ]:

"""
1-  İş Sağlığı ve Güvenliği (İSG) nedir?
Cevap: İş sağılığı ve güveliği (İŞG), işyerlerinin çalışanlar için sağlıklı, güvenli ve ergonomik hale getirilmesini sağlayan sistematik bir yaklaşımdır. Çalışanların fiziksel ve ruhsal sağlığının korunması, iş kazalarının ve meslek kaynaklı hastalıkların önlenmesini amaçlar ve yasa ile düzenlenir.


2-  Bakım ve onarım çalışmalarının yapılmaması hangi sonuçlara yol açabilir?
Cevap: Bakımda ve onarmada ihmal, ekipman arızasına, güvenlik sistemlerinin bozulmasına, iş kazaları ve ölümlere, üretim kaybıyla birlikte maliyetlerin artmasına yol acar.


3-  Bakım işlerinde en sık karşılaşılan kaza türü nedir?
Cevap: Metnin verdiği bilgiler ışığında, bakım-onarım sırasında en yaygın kaza tipi **ezilme/sıkıştırma/batma/kesme türündeki kazal ardır**. Bu tür kaza lara %1


4-  İş makineleri ve ekipmanları kaynaklı kazaların oranı nedir?
Cevap: Metindeki verilere göre, makinelerden kaynaklanan iş kazaların orani %11.33'tür (Tablo 3).


5-  İş ekipmanlarının bakım ve kontrolünü kimler yapmalıdır?
Cevap: Yetkili müh

6-  Elektrik kazalarının başlıca sebepleri nelerdir?
Cevap: Elektriki kazaların ana sebepları şunlardir:

1. **Yetersiz yalıtımlı elektrik tesisatı ve ekipmanları.**
2. **Arızalı elektrik panoları ve devre kesicileri.**
3. **Yanlış bağlantı ve topraklama hatası.**
4. **Aşırı yüklenmiş prizler ve kablolar.**
5. **Nemli ortamlarda elektrik kullanımı.**
6. **Bakım eksikliğinden kaynaklı arızalı ekipmanlar.**

7-  K faktörleri analizine göre iş kazalarına en çok etki eden faktör hangisidir?
Cevap: Metindeki verilere dayanarak, **"insan**" faktörünün iş kazasına en çok katkıda bulunduğu söylenebilir.

8-  İş kazalarının sebeplendirilmesinde insan faktörü neden bu kadar etkilidir?
Cevap: İşkazalarında insan faktörünün etkili olması, insanların hem sistem hatalarını tetiklemesi hem de kendi davranışları ile kazaları doğrudandırması nedeniyledir. İnsanların yorgunluk, dikkatsizlik, ihmalkârlık, deneyim eksikliğ, yanlış uygulama, iletişim sorunları ve acelenin getirdiği risk almalar gibi davranışsal faktörlere eğilimli olması kazalardaki başlıca etkendir.


9-  İşverenin periyodik bakım konusunda yasal sorumluluğu nedir?
Cevap: İş Sağılığı ve Guvenliği Yasası (4957) ve İş Ekıpmanlarının Kullanimında Saglık ve Güvelik Şarti Yöneticliği'ne göre iş veren, iş yerinde kullanılan tüm makina, teçızat ve ekıpmanların periyodi bakımının yapılması ve denetlenmesinden sorumludurtur.


10- Bakım ve onarım personeline özel risk değerlendirmesi yapılmasının gerekçesi nedir?
Cevap: Bakim ve onarim personeline ozel risk degeri ndirmesi yapilmasi, bu personellin is ekipmanlari uzerinde dogrudan müdahalesi nedeniyle maruz kala bilecekleri özgü tehliklerin belirlenme ve kontrol altina alinmasi icin gereklidir .


"""

"""
Modelin ayağa kalkma süresi : 1 dakika 30 saniye
Pdf yüklenmesi ile birlikte 10 sorunun sorulup gelen gelen cevapların toplam süresi : 9 dakika 30 saniye
"""

"""
| No | Soru Başlığı                          | Cevap PDF'de Var mı? | Cevap Doğru mu? | Açıklama                                                                                    |
| -- | ------------------------------------- | -------------------- | --------------- | ------------------------------------------------------------------------------------------- |
| 1  | İSG nedir?                            | ✅                    | ✅               | PDF'de tanım ve kapsam doğru şekilde anlatılmış (s.68).                                     |
| 2  | Bakım yapılmazsa ne olur?             | ✅                    | ✅               | Arıza, kazalar, üretim kaybı ve maliyet artışı vurgulanmış (s.70, 74).                      |
| 3  | En sık kaza türü?                     | ✅                    | ✅               | Ezilme/sıkışma/batma/kesme %37,1 ile en yüksek oranda (s.73).                               |
| 4  | Makine kaynaklı kaza oranı?           | ✅                    | ✅               | %11-14 arası olduğu belirtilmiş, %12 genel oran (s.72-73).                                  |
| 5  | Bakımı kim yapmalı?                   | ✅                    | ✅               | Yetkili mühendis, tekniker, teknisyen belirtilmiş (s.76).                                   |
| 6  | Elektrik kazalarının sebepleri?       | ✅                    | ✅               | Altı başlıkta detaylı nedenler anlatılmış (s.74).                                           |
| 7  | K faktörlerine göre en etkili faktör? | ✅                    | ✅               | İnsan faktörü en yüksek katkıya sahip: %40 direkt, %68 dolaylı (s.72).                      |
| 8  | İnsan faktörü neden etkili?           | ✅                    | ✅               | İhmal, yorgunluk, iletişimsizlik, dikkatsizlik vb. detaylı açıklanmış (s.78-79).            |
| 9  | İşverenin yasal sorumluluğu?          | ✅                    | ✅               | 6331 sayılı yasa ve yönetmelikler açıkça belirtilmiş (s.76-77).                             |
| 10 | Neden özel risk değerlendirmesi?      | ✅                    | ✅               | Avrupa Komisyonu rehberine dayanarak bakım personeline özel değerlendirme önerilmiş (s.79). |

"""

## 2. pdf için hazırlanmış test soruları ve sonuçları

In [ ]:
if __name__ == "__main__":
    main()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...


Cevap: Deriin öğrenme modelleri, çok sayıda gizli katman içeren yapay nöron ağlarını kullanırken, makıne öğrenmesi modelleri genellikle daha basit fonksiyonel formüllere dayanır. Deriin ögrenmede, özellikler otomatik çıkarılırken, maķine öğrenmesin de özellikler genellikle manuel olarak belirlenir.
--------------------------------------------------------------------------------
Sorunuz: PdM'de (Kestirimci Bakım) kullanılan öznitelik çıkarımı teknikleri nelerdir?


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Cevap: PCA (Temel Bileşenler Analizi), MakineÖğrenmesi algoritmalari (ANN vb.)
--------------------------------------------------------------------------------
Sorunuz: SVM algoritmasının PdM uygulamalarında dezavantajlarından biri nedir?


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Cevap: SVM, PdM’de yüksek boyutluluklu veri setleriyle çalışırken düşük performans gösterebilir ve hesaplamalar yavaşlayabilir.
--------------------------------------------------------------------------------
Sorunuz: çık


In [ ]:

"""
1-  Kestirimci bakımın temel amacı nedir?
Cevap: Potansiyel makineler arızlarını öncedنn tahmin etme ve planlı bakımlarla beklenmeyen duruşları önlemek.


2-  Kestirimci bakım hangi teknolojilerin gelişmesiyle önem kazanmıştır?
Cevap: KESTİRMECİ BAKIM, SENSÖR TEKNOLOJİLERİNİN GELİŞMESİ VE MAKİNE ÖĞRENMESİ ALGORİTMALARININ UYGULANABİLİRLİĞİ SAYESİNDE ÖNEM KAZANMIŞTIR.


3-  Makine öğrenmesi, yapay zekâ yaklaşımlarının hangi alt grubudur?
Cevap: Yapak zekanın bir **alt dalıdır**.


4-  Literatürde kestirimci bakım stratejileri kaça ayrılır?
Cevap: Metne göre kestiricci bakım alt kategorilere ayrılmamış.


5-  Korelasyon analizi kestirimci bakımda nasıl kullanılır?
Cevap: Korelâsyon analîzi, kestîrîmcî bakîmda, makîne parçalarının durumunu gösterebilecek çeşitli sensör ölçümlerinden elde edilen veriler arasında anlamlı ilişkileri bulmak amacıyla kullanılır. Pozitif veya negatif korelasyonun olup olmadığını ve şidd


6-  Makine öğrenmesinde denetimli öğrenmenin özelliği nedir?
Cevap: Denetimlö öğrenmede, model **eğitim sırasında hem giriş hem de doğru çıktı verisiyle beslenir**. Bu sayede model, girdileri çıktıyla eşleştiren bir ilişki kurar ve yeni verileri sınıflandırmak veya tahmin etmek için bu ilişkiye dayanır.


7-  Karar ağacı algoritmasında sınıflandırma nasıl yapılır?
Cevap: Karars ağacı sınıflandırmada, veri seti iteratif olarak bölünerek her dalın belirli sınıfı temsil ettiği bir hiyerarşik yapı oluşturulur. Her adımda, en iyi özellik seçilip veri seti bu özellikteki eşik değere göre iki alt kümeye ayrılır; bu işlem, yaprak düğümlerde sınıf dağılımına göre sonlanır.


8-  Derin öğrenme algoritmalarının makine öğrenmesi algoritmalarına göre farkı nedir?
Cevap: Deriin öğrenme modelleri, çok sayıda gizli katman içeren yapay nöron ağlarını kullanırken, makıne öğrenmesi modelleri genellikle daha basit fonksiyonel formüllere dayanır. Deriin ögrenmede, özellikler otomatik çıkarılırken, maķine öğrenmesin de özellikler genellikle manuel olarak belirlenir.


9-  PdM'de (Kestirimci Bakım) kullanılan öznitelik çıkarımı teknikleri nelerdir?
Cevap: PCA (Temel Bileşenler Analizi), MakineÖğrenmesi algoritmalari (ANN vb.)


10- SVM algoritmasının PdM uygulamalarında dezavantajlarından biri nedir?
Cevap: SVM, PdM’de yüksek boyutluluklu veri setleriyle çalışırken düşük performans gösterebilir ve hesaplamalar yavaşlayabilir.


"""

"""
Modelin ayağa kalkma süresi : 1 dakika 30 saniye
Pdf yüklenmesi ile birlikte 10 sorunun sorulup gelen gelen cevapların toplam süresi : 12 dakika 15 saniye
"""

"""
| Soru No | Doğruluk        | Yorum                                                                                                         | Puan (10 üzerinden) |
| ------- | --------------- | ------------------------------------------------------------------------------------------------------------- | ------------------- |
| 1       | ✅ Doğru         | Teknik olarak net ve doğru; ifade açık.                                                                       | 9                   |
| 2       | ✅ Doğru         | Sensör teknolojileri ve ML algoritmaları vurgusu yerinde.                                                     | 9                   |
| 3       | ✅ Doğru         | “Alt dalı” tanımı sade ama doğru. Daha detay eklenebilir.                                                     | 8                   |
| 4       | 🟡 Eksik        | Metinde sınıflandırma olmayabilir ama genel literatürde üç ana strateji bulunur. Soruya tam cevap verilmemiş. | 5                   |
| 5       | ✅ Doğru         | Korelasyon analizinin PdM'de nasıl kullanıldığını güzel açıklamış. Son cümle kesilmiş.                        | 8                   |
| 6       | ✅ Doğru         | Denetimli öğrenme çok net ve doğru tanımlanmış.                                                               | 10                  |
| 7       | ✅ Doğru         | Karar ağacı algoritması detaylı ve akademik biçimde açıklanmış.                                               | 10                  |
| 8       | ✅ Doğru         | Derin öğrenme ve makine öğrenmesi farkı teknik ve anlamlı bir dille açıklanmış.                               | 10                  |
| 9       | 🟡 Kısmen doğru | PCA doğru örnek; ANN, öznitelik çıkarımından çok sınıflama için kullanılır. Bazı kavramlar karışmış.          | 6                   |
| 10      | ✅ Doğru         | SVM’in yüksek boyutlarda yavaş çalışması bilinen bir zayıflık, teknik olarak doğru.                           | 9                   |


"""

## 3. pdf için hazırlanmış test soruları ve sonuçları

In [ ]:
if __name__ == "__main__":
    main()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...


Cevap: **Önleme maliyeti, Değerlendirmeyi maliyeti , İç başarıssizlik maliyeti ve Dış başarısısizlik maliyet.**
--------------------------------------------------------------------------------
Sorunuz: Genetik algoritmaların yapay zekâdaki rolü nedir?


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Cevap: Geneik algoritmalar yapay seleksiyon ve evrimini simüle ederek karmaşik problemlerde optimum çözümlere ulaşmada kullanılan bir optimizasyondur.
--------------------------------------------------------------------------------
Sorunuz: Kalite 4.0’ın geleneksel kalite anlayışına göre sağladığı farklılık nedir?


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Cevap: Kalité 4.,0, **veriden beslenen, otomatik ve proaktif** bir kalite yönetimi yaklaşımı sunarken, geleneke! kalite yönerimi **insan merkezli, reaktif ve veri toplamayı sınırlayan** bir yapıdadır.
--------------------------------------------------------------------------------
Sorunuz: çık


In [ ]:

"""
1-  Yapay zekâ teknolojilerinin işletmelere sağladığı temel fayda nedir?
Cevap: Ya pay ze kâ tekno lo jileri iş let melere **üretim verimliliğini artırarak**, **mali yetleri düş ürerek** ve  **karlılığı yükseltmek** te temel fay da sağ lar.


2-  Yapay zekâ hangi alanlarda uygulanmaktadır?
Cevap: Sağlık, ulaştırma, eğitim , e-ticaretin yanı sıra üretim sektöründe (Endüstrı 4,0), büyük ver analitiğinde ve Nesneler İnternetinde.


3-  Yapay zekâ nedir?
Cevap: Ya p ay zek ânın amacı, insan benzer i zek asiyel faaliyetleri gerçekleştiren bilgisayar sistemleri geliştirmektir. Bu faaliyetler arasında öğrenme , problem çöz me, karar alma ve dil işleme bulunur.


4-  Kalite maliyetleri neden önemlidir?
Cevap: Kalité maliyetleriyle ilgili detaylı bilgi vermek yerine, genel olarak şunları söyleyebiliriz: Kalitesi düşük ürünler, işyerine ekstra maliyet getirir. Bu maliyet, üretilen ürünlerin tekrar yapılmasından kaynaklanır. Müşterilerin memnun olmaması durumunda ise şirketin itibarı zarar görür ve bu da uzun vadenli gelir kaybına yol açar. Dolayısıyla, kaliteli ürün üretmek, işyeri için daha ekonomik ve sürdürülebililir bir seçenektir.


5-  Kalite 4.0 nedir?
Cevap: Kalitè 4.,0, dijtal dönüşümü kalite yönetime entegre eden bir yaklaşım olup, yapak zeka (YZ), büyük veri, IoT ve bulüt bilişim teknolojilerini kullanarak üretim ve hizmet süreçlerinin kalitesini artıran bir sistemdir.


6-  Kestirimci bakımın amacı nedir?
Cevap: Ekipman arıza riski taşıyan bileşenleri erken tespit ederek planlı bakımların gerçekleştirilmesi ve üretimi aksatacak beklenmeyen arızalara engel olmaktır.


7-  Yapay zekâ tabanlı kalite yönetim sistemlerinin avantajlarından biri nedir?
Cevap: **Erken hata tespiti** ve **önleme**.


8-  PAF modeli kapsamında kalite maliyetleri hangi dört başlık altında toplanır?
Cevap: **Önleme maliyeti, Değerlendirmeyi maliyeti , İç başarıssizlik maliyeti ve Dış başarısısizlik maliyet.**


9-  Genetik algoritmaların yapay zekâdaki rolü nedir?
Cevap: Geneik algoritmalar yapay seleksiyon ve evrimini simüle ederek karmaşik problemlerde optimum çözümlere ulaşmada kullanılan bir optimizasyondur.


10- Kalite 4.0’ın geleneksel kalite anlayışına göre sağladığı farklılık nedir?
Cevap: Kalité 4.,0, **veriden beslenen, otomatik ve proaktif** bir kalite yönetimi yaklaşımı sunarken, geleneke! kalite yönerimi **insan merkezli, reaktif ve veri toplamayı sınırlayan** bir yapıdadır.



"""

"""
Modelin ayağa kalkma süresi : 1 dakika 30 saniye
Pdf yüklenmesi ile birlikte 10 sorunun sorulup gelen gelen cevapların toplam süresi : 12 dakika 30 saniye
"""

"""
| Soru No | Doğruluk        | Yorum                                                                                                                                             | Puan (10 üzerinden) |
| ------- | --------------- | ------------------------------------------------------------------------------------------------------------------------------------------------- | ------------------- |
| 1       | ✅ Doğru         | Verimlilik, maliyet ve kârlılık vurgusu gayet yerinde ve doğru.                                                                                   | 9                   |
| 2       | ✅ Doğru         | Uygulama alanları doğru belirtilmiş, kısa ama yeterli.                                                                                            | 9                   |
| 3       | ✅ Doğru         | Tanım doğru ve teknik; yapay zekâ faaliyetlerini güzel özetlemiş.                                                                                 | 9                   |
| 4       | ✅ Doğru         | Kalite maliyetinin önemi güzel açıklanmış, ifade uzun ama etkili.                                                                                 | 9                   |
| 5       | ✅ Doğru         | Kalite 4.0'ın temel teknolojileri ve amacı iyi yansıtılmış. Küçük yazım hataları var.                                                             | 8                   |
| 6       | ✅ Doğru         | PdM’nin amacı net bir şekilde açıklanmış.                                                                                                         | 10                  |
| 7       | ✅ Doğru         | Cevap kısa ama etkili; doğru avantaj vurgulanmış.                                                                                                 | 8                   |
| 8       | 🟡 Kısmen doğru | Doğru maddeler verilmiş ama yazım hataları ciddi (örneğin “başarısısizlik”).                                                                      | 6                   |
| 9       | ✅ Doğru         | Genetik algoritmaların işlevi doğru anlatılmış. Küçük imla hatası var (“optimizasyondur” yerine “optimizasyon algoritmasıdır” daha uygun olurdu). | 8                   |
| 10      | ✅ Doğru         | Kalite 4.0 ve geleneksel kalite farkları net biçimde karşılaştırılmış. Küçük yazım hataları var ama içerik güçlü.                                 | 9                   |


"""

# Sonuç

3 Farklı PDF den 10'ar soru ile sistemi test ettim. Bu testlerde daha önce kullandığım emrecanın embeeding modelini kullandım ayrıca reranker modelinin hızı oldukça etkilediğini fark ettiğim için çıkardım ayrıca embeeding modelinin benzerlik skoruna göre artık ilk 5 cevabı getiriyorum. Daha önce geliştirme kodları 2 çalışmamdan en iyi performansı gösteren iki LLM modelini burada detaylı test etmiş bulundum. Her iki modelin ürettiği cevaplarda hala yazım yanlışları mevcut. Ek olarak hala çok hızlı olduğu söylenemez.

**Aşağıda sonuç tablosu mevcut!**


\begin{array}{|l|c|c|c|c|}
\hline
\textbf{Model} & \textbf{Doğru} & \textbf{Kısmen Doğru} & \textbf{Yanlış} & \textbf{Başarı Oranı (\%)} \\
\hline
\text{KOCDIGITAL/Kocdigital-LLM-8b-v0.1} & 10 & 7 & 13 & 45 \\
\hline
\text{ytu-ce-cosmos/Turkish-Gemma-9b-v0.1} & 27 & 3 & 0 & 95 \\
\hline
\end{array}


# ytu-ce-cosmos/Turkish-Gemma-9b-v0.1 + Çoklu PDF SİSTEMİ

In [ ]:
# GEREKLİ KÜTÜPHANELER
!pip install transformers torch sentence-transformers faiss-cpu PyPDF2
import torch
import faiss
import re
import numpy as np
import PyPDF2
import io
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
from google.colab import files

torch.set_float32_matmul_precision('high')


class TurkishPDFQA_Fast:
    def __init__(self):
        # LLM yükleniyor
        self.llm_model_name = "ytu-ce-cosmos/Turkish-Gemma-9b-v0.1"
        self.tokenizer = AutoTokenizer.from_pretrained(self.llm_model_name)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        self.model = AutoModelForCausalLM.from_pretrained(
            self.llm_model_name,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto" if torch.cuda.is_available() else None,
            low_cpu_mem_usage=True
        )
        self.device = next(self.model.parameters()).device

        # Embedding modeli
        self.embed_model = SentenceTransformer("emrecan/bert-base-turkish-cased-mean-nli-stsb-tr")

        self.pdf_chunks = []
        self.embeddings = None
        self.index = None

    def extract_text_from_pdf(self, pdf_file):
        print("PDF okunuyor...")
        reader = PyPDF2.PdfReader(pdf_file)
        full_text = ""
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                full_text += page_text + "\n"
        clean = re.sub(r'\s+', ' ', full_text)
        clean = re.sub(r'[^\w\s.,!?;:()\-]', '', clean)
        self._split_and_index(clean.strip())
        print("PDF'den metin çıkarıldı ve embedding index oluşturuldu.")

    def extract_text_from_multiple_pdfs(self, pdf_files):
        """Birden fazla PDF dosyasından metin çıkarır ve tek index oluşturur"""
        print(f"{len(pdf_files)} PDF dosyası okunuyor...")
        all_texts = []

        for i, pdf_file in enumerate(pdf_files, 1):
            print(f"PDF {i}/{len(pdf_files)} işleniyor...")
            reader = PyPDF2.PdfReader(pdf_file)
            full_text = ""
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    full_text += page_text + "\n"
            clean = re.sub(r'\s+', ' ', full_text)
            clean = re.sub(r'[^\w\s.,!?;:()\-]', '', clean)
            all_texts.append(clean.strip())

        # Tüm metinleri birleştir
        combined_text = "\n\n--- YENİ DÖKÜMAN ---\n\n".join(all_texts)
        self._split_and_index(combined_text)
        print("Tüm PDF'lerden metin çıkarıldı ve tek embedding index oluşturuldu.")

    def _split_and_index(self, text, chunk_size=500, stride=100):
        print("Metin token bazlı parçalara bölünüyor ve embed ediliyor...")
        tokens = self.tokenizer.encode(text, add_special_tokens=False)
        self.pdf_chunks = []
        start = 0
        while start < len(tokens):
            end = min(start + chunk_size, len(tokens))
            chunk_tokens = tokens[start:end]
            chunk_text = self.tokenizer.decode(chunk_tokens, clean_up_tokenization_spaces=True)
            self.pdf_chunks.append(chunk_text)
            start += chunk_size - stride

        self.embeddings = self.embed_model.encode(self.pdf_chunks, convert_to_numpy=True, show_progress_bar=True)
        self.index = faiss.IndexFlatL2(self.embeddings.shape[1])
        self.index.add(self.embeddings)

    def ask_question(self, question, top_k=5):
        question_embedding = self.embed_model.encode([question], convert_to_numpy=True)
        D, I = self.index.search(question_embedding, top_k)
        top_chunks = [self.pdf_chunks[i] for i in I[0]]

        answers = []
        for chunk in top_chunks:
            prompt = f"""Metin: {chunk}\n\nSoru: {question}\n\nCevap:"""
            inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(self.device)
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=100,
                    temperature=0.4,
                    top_p=0.95,
                    top_k=40,
                    do_sample=False,
                    repetition_penalty=1.1,
                    no_repeat_ngram_size=3,
                    pad_token_id=self.tokenizer.eos_token_id,
                    eos_token_id=self.tokenizer.eos_token_id,
                )
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            answers.append(response.strip())

        fusion_prompt = "\n".join([f"Parça {i+1} Cevap: {ans}" for i, ans in enumerate(answers)])
        fusion_prompt += f"""

Sadece yukarıdaki cevaplara dayalı teknik ve doğru bir Türkçe cevap ver.
Genel açıklamalardan, tahminlerden ve konu dışı ifadelerden kaçın. Sadece doğrudan sorunun teknik cevabını ver.

SORU: {question}

CEVAP:"""

        fusion_inputs = self.tokenizer(fusion_prompt, return_tensors="pt", truncation=True, padding=True).to(self.device)
        with torch.no_grad():
            final_output = self.model.generate(
                **fusion_inputs,
                max_new_tokens=150,
                temperature=0.4,
                top_p=0.95,
                top_k=40,
                do_sample=False,
                repetition_penalty=1.1,
                no_repeat_ngram_size=3,
                pad_token_id=self.tokenizer.eos_token_id,
                eos_token_id=self.tokenizer.eos_token_id,
            )
        final_answer = self.tokenizer.decode(final_output[0], skip_special_tokens=True)
        return final_answer.split("CEVAP:")[-1].strip()


# ================= ANA PROGRAM =================
def main():
    qa = TurkishPDFQA_Fast()

    print("PDF yükleme seçenekleri:")
    print("1 - Tek PDF yükle")
    print("2 - Birden fazla PDF yükle")

    choice = input("Seçiminizi yapın (1 veya 2): ").strip()

    if choice == "1":
        print("Lütfen bir PDF yükleyin...")
        uploaded = files.upload()
        if uploaded:
            file_name = list(uploaded.keys())[0]
            with open(file_name, 'rb') as f:
                qa.extract_text_from_pdf(f)
        else:
            print("PDF yüklenemedi.")
            return

    elif choice == "2":
        pdf_files = []
        print("Birden fazla PDF yükleme modu:")
        print("Her seferinde bir PDF seçin. Bitirmek için 'hayır' yazın.")

        while True:
            more_files = input("PDF yüklemek istiyor musunuz? (evet/hayır): ").strip().lower()
            if more_files in ['hayır', 'h', 'no', 'n']:
                break

            print(f"PDF {len(pdf_files) + 1} yükleniyor...")
            uploaded = files.upload()
            if uploaded:
                for file_name in uploaded.keys():
                    with open(file_name, 'rb') as f:
                        pdf_files.append(f.read())
                print(f"PDF {len(pdf_files)} başarıyla eklendi!")
            else:
                print("PDF yüklenemedi, devam ediliyor...")

        if pdf_files:
            # BytesIO nesneleri oluştur
            pdf_file_objects = [io.BytesIO(pdf_data) for pdf_data in pdf_files]
            qa.extract_text_from_multiple_pdfs(pdf_file_objects)
        else:
            print("Hiç PDF yüklenmedi.")
            return

    else:
        print("Geçersiz seçim.")
        return

    print("\nPDF(ler) yüklendi! Artık soru sorabilirsiniz. Çıkmak için 'çık' yazın.\n")
    while True:
        q = input("Sorunuz: ")
        if q.lower() in ['çık', 'exit', 'quit']:
            break
        answer = qa.ask_question(q)
        print("\nCevap:", answer)
        print("-" * 80)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 100.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 25.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink

In [ ]:
if __name__ == "__main__":
    main()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...


Cevap: Yapak zeka, kalite denetim fonksiyonlarını standartlaştırıp verimlileştirirken, aynı zamanda üretim sürecini optimize ederek maliyet düşürme sağlar.
--------------------------------------------------------------------------------
Sorunuz: PAF modeli kapsamında kalite maliyetleri hangi dört başlık altında toplanır?


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Cevap: Önleme maliyeti , değerlendir me maliyeti  , iç hata maliyeti ve dış hata maliye ti .
--------------------------------------------------------------------------------
Sorunuz: Endüstride iş kazalarını önlemek ve kaliteyi artırmak için periyodik bakım, kestirimci bakım ve yapay zekâ tabanlı sistemlerin birlikte kullanımı nasıl bir etki yaratır?


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Cevap: Bu üç yaklaşımların birlikte uygulanması, iş kazası riskini azaltırken kaliteyi arttırır. 

Periyodik bakar, rutin kont roller ve temizlikle olası arızalara karşı önlem alarak güvenliği artırır ve ekipman ömrünü uzatır. K estirim ci bakım, sensör verilerini analiz ederek arıza ihtimalini öngörüyor ve bakımın tam zamanında yapılmasını sağlayarak ani arızalanmayı önler. Ya pay ze kâ tabanl ı sistemler ise, geniş veri kümeleri üzerinden desenleri tanımlayarak operatör hatalarının önüne geçer, üre tim süreçlerini optimize eder ve kalite kontrolünde daha
--------------------------------------------------------------------------------
Sorunuz: çık


In [ ]:
"""
1. PDF test soruları ve gelen cevaplar:

-  K faktörleri analizine göre iş kazalarına en çok etki eden faktör hangisidir?
Cevap: Metindeki verilere dayanarak, **"insan**" faktörünün iş kazasına en çok katkıda bulunduğu söylenebilir.


-  İş kazalarının sebeplendirilmesinde insan faktörü neden bu kadar etkilidir?
Cevap: İşkazalarında insan faktörünün etkili olması, insanların hem sistem hatalarını tetiklemesi hem de kendi davranışları ile kazaları doğrudandırması nedeniyledir. İnsanların yorgunluk, dikkatsizlik, ihmalkârlık, deneyim eksikliğ, yanlış uygulama, iletişim sorunları ve acelenin getirdiği risk almalar gibi davranışsal faktörlere eğilimli olması kazalardaki başlıca etkendir.


-  İşverenin periyodik bakım konusunda yasal sorumluluğu nedir?
Cevap: İş Sağılığı ve Guvenliği Yasası (4957) ve İş Ekıpmanlarının Kullanimında Saglık ve Güvelik Şarti Yöneticliği'ne göre iş veren, iş yerinde kullanılan tüm makina, teçızat ve ekıpmanların periyodi bakımının yapılması ve denetlenmesinden sorumludurtur.





2. PDF test soruları ve gelen cevaplar:

-  Karar ağacı algoritmasında sınıflandırma nasıl yapılır?
Cevap: Kararar ağacı algortiması, veri seti üzerinde tekrarlayan bölme operasyonları yaparak sınıflandıma gerçekleştirir:

- **Başlangıç**: Tüm veri örneği tek bir kökünde bulunur.
- **Bölme Kriteri**: En uygun özellik seçilerek veri seti alt kümelere ayrılır (örneğin, "Boy" özelliği >180 cm ise sol dal, <=18o cm ise sağ dal).
- *Terminal Düğümler*: Bölme durdurma koşulu sağlandığında (örnekler aynı sınıfa ait veya maksimum derinlik ulaşıldığında) terminal düğüm oluşturulur ve bu düğüm sınıf etiketimi taşır.

-  Derin öğrenme algoritmalarının makine öğrenmesi algoritmalarına göre farkı nedir?
Cevap: Der

-  PdM'de (Kestirimci Bakım) kullanılan öznitelik çıkarımı teknikleri nelerdir?
Cevap: 1.**Zaman Alanı Özellikleri**:
   – Ortalama


2.**Frekans Domain Özellikleri (Spektrum Analizi)**:



3.**Özellik Filtreleme ve Seçimi**:




3. PDF test soruları ve gelen cevaplar:

-  Kestirimci bakımın amacı nedir?
Cevap: Ekipman arizalarını ö nceden tespit edebilmek ve böylece planlı bakımlar yaparak beklenmeyen arızalan ve bunların yol açtığı problemleri önlemek.


-  Yapay zekâ tabanlı kalite yönetim sistemlerinin avantajlarından biri nedir?
Cevap: Yapak zeka, kalite denetim fonksiyonlarını standartlaştırıp verimlileştirirken, aynı zamanda üretim sürecini optimize ederek maliyet düşürme sağlar.


-  PAF modeli kapsamında kalite maliyetleri hangi dört başlık altında toplanır?
Cevap: Önleme maliyeti , değerlendir me maliyeti  , iç hata maliyeti ve dış hata maliye ti .


ortak soru : "Endüstride iş kazalarını önlemek ve kaliteyi artırmak için periyodik bakım, kestirimci bakım ve yapay zekâ tabanlı sistemlerin birlikte kullanımı nasıl bir etki yaratır?"
cevap : Cevap: Bu üç yaklaşımların birlikte uygulanması, iş kazası riskini azaltırken kaliteyi arttırır.

Periyodik bakar, rutin kont roller ve temizlikle olası arızalara karşı önlem alarak güvenliği artırır ve ekipman ömrünü uzatır. K estirim ci bakım, sensör verilerini analiz ederek arıza ihtimalini öngörüyor ve bakımın tam zamanında yapılmasını sağlayarak ani arızalanmayı önler. Ya pay ze kâ tabanl ı sistemler ise, geniş veri kümeleri üzerinden desenleri tanımlayarak operatör hatalarının önüne geçer, üre tim süreçlerini optimize eder ve kalite kontrolünde daha



"""


"""

| Soru                 | Değerlendirme   |
| -------------------- | --------------- |
| K faktörleri sorusu  | ✅ Doğru         |
| İnsan faktörü etkisi | ✅ Doğru         |
| Yasal sorumluluk     | ✅ Doğru         |
| Karar ağacı          | ✅ Doğru         |
| Derin öğrenme farkı  | ❌ Eksik         |
| PdM öznitelikleri    | ✅ Doğru (eksik) |
| PdM amacı            | ✅ Doğru         |
| Yapay zekâ & kalite  | ✅ Doğru         |
| PAF modeli           | ✅ Doğru         |


Ortak Soru Değerlendirmesi (GPT'ye yorumlattım cevabın doğruluğunu) :

| Ölçüt                                                                              | Puan (0–1) | Açıklama                                                                                              |
| ---------------------------------------------------------------------------------- | ---------- | ----------------------------------------------------------------------------------------------------  |
| 1. Periyodik bakımın temel işlevi doğru tanımlanmış mı?                            | ✅ 1        | "Rutin kontrollerle arızalara karşı önlem" ifadesi yerinde.                                          |
| 2. Periyodik bakımın **güvenlik etkisi** belirtilmiş mi?                           | ✅ 1        | "Güvenliği artırır" demesi doğrudur.                                                                 |
| 3. Periyodik bakımın **ekipman ömrüne katkısı** yer alıyor mu?                     | ✅ 1        | "Ekipman ömrünü uzatır" ifadesi yerinde.                                                             |
| 4. Kestirimci bakımın amacı (veri analiziyle arıza tahmini) var mı?                | ✅ 1        | "Sensör verilerini analiz ederek arıza ihtimalini öngörür" ifadesi yerinde.                          |
| 5. Kestirimci bakımın **zamanında müdahale ve arıza önleme** etkisi anlatılmış mı? | ✅ 1        | "Bakımın tam zamanında yapılmasını sağlar" ifadesi doğru.                                            |
| 6. Yapay zekâ sistemlerinin veri analiziyle öğrenme yaptığı belirtilmiş mi?        | ✅ 1        | "Geniş veri kümeleri üzerinden desenleri tanımlar" cümlesi doğru.                                    |
| 7. Yapay zekânın **operatör hatalarını azaltma** işlevi belirtilmiş mi?            | ✅ 1        | "Operatör hatalarının önüne geçer" ifadesi yerinde.                                                  |
| 8. Yapay zekânın **kalite kontrolü ve üretim optimizasyonu** etkisi var mı?        | ✅ 1        | "Üretim süreçlerini optimize eder ve kalite kontrolünde..." kısmı yeterli.                           |
| 9. Yapay zekânın **maliyet veya kalite maliyetine** etkisi belirtilmiş mi?         | ❌ 0        | Eksik; 3. PDF’te önemli olan bu katkı ifade edilmemiş.                                               |
| 10. Tüm bileşenlerin **entegrasyonunun sistematik faydası** anlatılmış mı?         | ➖ 0.5      | "İş kazası riskini azaltır, kaliteyi artırır" denmiş ama bütünsel sistem etkisi zayıf ifade edilmiş. |



"""

In [ ]:
!nvidia-smi

Wed Jul  9 09:47:23 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P0             44W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----